In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets

import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

In [3]:
#model
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), # optimization
             loss='sparse_categorical_crossentropy', # loss function
             metrics=['accuracy']) # metrics / accuracy

In [5]:
# Data Preprocess

train_paths = sorted(glob('/Users/sengjeawang/Desktop/data/dataset/cifar/train/*.png'))[:1000]
test_paths = sorted(glob('/Users/sengjeawang/Desktop/data/dataset/cifar/test/*.png'))[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [7]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)

In [8]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    image = tf.cast(image, tf.float32) / 255.
    
    label = get_label(path)
    return image, label

In [9]:
def image_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [12]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

In [13]:
# Callbacks

logdir = os.path.join('/Users/sengjeawang/Desktop/data/logs', datetime.now().strftime("%Y%m%d-%H%M%S"))

In [14]:
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=logdir,
    write_graph=True,
    write_images=True,
    histogram_freq=1
)

In [15]:
%tensorboard --logdir=/Users/sengjeawang/Desktop/data/logs

Reusing TensorBoard on port 6006 (pid 6236), started 0:51:38 ago. (Use '!kill 6236' to kill it.)

In [17]:
#Training

steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard]
)

/Users/sengjeawang/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
31/31 [==============================] - 3s 82ms/step - loss: 2.3055 - accuracy: 0.1054 - val_loss: 2.2816 - val_accuracy: 0.1381
Epoch 2/10
31/31 [==============================] - 2s 77ms/step - loss: 2.2055 - accuracy: 0.1764 - val_loss: 2.1259 - val_accuracy: 0.2127
Epoch 3/10
31/31 [==============================] - 2s 78ms/step - loss: 2.1328 - accuracy: 0.2044 - val_loss: 2.0365 - val_accuracy: 0.2319
Epoch 4/10
31/31 [==============================] - 2s 76ms/step - loss: 2.0603 - accuracy: 0.2448 - val_loss: 1.9660 - val_accuracy: 0.2550
Epoch 5/10
31/31 [==============================] - 2s 77ms/step - loss: 1.9984 - accuracy: 0.2386 - val_loss: 2.0113 - val_accuracy: 0.2248
Epoch 6/10
31/31 [==============================] - 2s 76ms/step - loss: 1.9914 - accuracy: 0.2572 - val_loss: 1.9544 - val_accuracy: 0.2692
Epoch 7/10
31/31 [==============================] - 2s 77ms/step - loss: 1.9125 - accuracy: 0.2779 - val_loss: 1.9424 - val_accuracy: 0.2933
Epoch 8/10
31

In [18]:
# Lambda Callback

# https://www.tensorflow.org/tensorboard/r2/image_summaries

import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [19]:
test_images, test_labels = next(iter(test_dataset))  # Confusion Matrix 그릴 때 필요한 Test Image


def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [20]:
# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [22]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard, cm_callback]
)

/Users/sengjeawang/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
31/31 [==============================] - 3s 77ms/step - loss: 1.8521 - accuracy: 0.3089 - val_loss: 1.8248 - val_accuracy: 0.3095


<ipython-input-18-27db1b8c90c7>:45: RuntimeWarning: invalid value encountered in true_divide
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)


Epoch 2/10
31/31 [==============================] - 2s 74ms/step - loss: 1.8250 - accuracy: 0.3151 - val_loss: 1.8842 - val_accuracy: 0.3004
Epoch 3/10
31/31 [==============================] - 2s 76ms/step - loss: 1.7995 - accuracy: 0.3110 - val_loss: 1.8119 - val_accuracy: 0.3337
Epoch 4/10
31/31 [==============================] - 2s 75ms/step - loss: 1.7264 - accuracy: 0.3574 - val_loss: 1.8939 - val_accuracy: 0.3044
Epoch 5/10
31/31 [==============================] - 2s 73ms/step - loss: 1.7333 - accuracy: 0.3616 - val_loss: 1.7614 - val_accuracy: 0.3196
Epoch 6/10
31/31 [==============================] - 2s 72ms/step - loss: 1.7147 - accuracy: 0.3605 - val_loss: 1.8638 - val_accuracy: 0.3438
Epoch 7/10
31/31 [==============================] - 2s 74ms/step - loss: 1.6937 - accuracy: 0.3667 - val_loss: 1.7403 - val_accuracy: 0.3579
Epoch 8/10
31/31 [==============================] - 2s 74ms/step - loss: 1.6261 - accuracy: 0.3905 - val_loss: 1.7833 - val_accuracy: 0.3407
Epoch 9/10
31

In [39]:
# Expert

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [46]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [47]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [48]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [49]:
logdir = os.path.join('logs', datetime.now().strftime("%Y%m%d-%H%M%S"))

In [50]:
file_writer = tf.summary.create_file_writer(logdir)

In [51]:
for epoch in range(num_epochs):
    for step, (images, labels) in enumerate(train_dataset):
        train_step(images, labels)
        
    with file_writer.as_default():
        tf.summary.image('input_image', images, step=step+(step*epoch))
        tf.summary.scalar('train_loss', train_loss.result(), step=step+(step*epoch))
        tf.summary.scalar('train_accuracy', train_accuracy.result(), step=step+(step*epoch))
        
    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

KeyboardInterrupt: 